<a href="https://colab.research.google.com/github/arumlim/eileen/blob/master/05_01_%E1%84%80%E1%85%B3%E1%86%B7%E1%84%8B%E1%85%AD%E1%86%BC%E1%84%91%E1%85%A9%E1%84%90%E1%85%B3%E1%84%91%E1%85%A9%E1%86%AF%E1%84%85%E1%85%B5%E1%84%8B%E1%85%A9_%EC%B5%9C%EC%A0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 포트폴리오 보유 비중 최적화

In [ ]:
!pip install finance-datareader

In [ ]:
import FinanceDataReader as fdr

In [ ]:
# 한국거래소 상장종목 전체
KRX_list = fdr.StockListing('KRX')
KRX_list.head()

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
0,060310,KOSDAQ,3S,전자부품 제조업,반도체 웨이퍼 캐리어,2002-04-23,03월,"박종익, 김세완 (각자 대표이사)",http://www.3sref.com,서울특별시
1,095570,KOSPI,AJ네트웍스,산업용 기계 및 장비 임대업,"렌탈(파렛트, OA장비, 건설장비)",2015-08-21,12월,박대현,http://www.ajnet.co.kr,서울특별시
2,006840,KOSPI,AK홀딩스,기타 금융업,지주사업,1999-08-11,12월,"채형석, 이석주(각자 대표이사)",http://www.aekyunggroup.co.kr,서울특별시
3,054620,KOSDAQ,APS홀딩스,기타 금융업,인터넷 트래픽 솔루션,2001-12-04,12월,정기로,http://www.apsholdings.co.kr,경기도
4,265520,KOSDAQ,AP시스템,특수 목적용 기계 제조업,디스플레이 제조 장비,2017-04-07,12월,김영주,http://www.apsystems.co.kr,경기도


In [ ]:
cor_list = ['메리츠화재','효성','삼성전자','한양이엔지','맥쿼리인프라','쌍용C&E']
종목비중 = np.array([0.2,0.2,0.15,0.15,0.15,0.15])

In [ ]:
cor_df = KRX_list[KRX_list['Name'].isin(cor_list)]
cor_df.head()

,Symbol,Market,Name,Sector,Industry,ListingDate,SettleMonth,Representative,HomePage,Region
2713,088980,KOSPI,맥쿼리인프라,신탁업 및 집합투자업,증권투자회사,2006-03-15,12월,맥쿼리자산운용(주),http://www.macquarie.com/kr/kr/mkif/index.html,서울특별시
2768,000060,KOSPI,메리츠화재,보험업,"손해보험(화재,해상,운송보험)",1956-07-02,12월,김용범,http://www.meritzfire.com,서울특별시
4074,005930,KOSPI,삼성전자,통신 및 방송 장비 제조업,"IMT2000 서비스용 동기식 기지국,교환국장비,데이터단말기,동영상휴대폰,핵심칩,반...",1975-06-11,12월,"김기남, 김현석, 고동진",http://www.sec.co.kr,경기도
4484,003410,KOSPI,쌍용C&E,"시멘트, 석회, 플라스터 및 그 제품 제조업",시멘트(크링카),1975-05-03,12월,"홍사승, 이현준 (각자)대표집행임원",http://www.ssangyongcement.co.kr,서울특별시
6987,045100,KOSDAQ,한양이엔지,"건축기술, 엔지니어링 및 관련 기술 서비스업",반도체배관설비,2000-08-31,12월,"김형육,김덕림(공동대표이사)",http://www.hanyangkorea.com,경기도


In [ ]:
cor_price_data = pd.DataFrame()
for idx, values in cor_df.iterrows():
  symbol = values[0]
  name = values[2]
  df = fdr.DataReader(symbol, '2018')
  df['cor_Name'] = name
  cor_price_data = pd.concat([cor_price_data,df])
cor_price_data

,Open,High,Low,Close,Volume,Change,cor_Name
Date,,,,,,,
2018-01-02,8260,8270,8210,8220,340631,-0.003636,맥쿼리인프라
2018-01-03,8220,8260,8210,8230,346181,0.001217,맥쿼리인프라
2018-01-04,8220,8230,8150,8180,708016,-0.006075,맥쿼리인프라
2018-01-05,8210,8210,8130,8170,673049,-0.001222,맥쿼리인프라
2018-01-08,8170,8170,8120,8140,549485,-0.003672,맥쿼리인프라
...,...,...,...,...,...,...,...
2021-11-22,96300,97600,95800,97100,34202,0.007261,효성
2021-11-23,96800,98000,96400,97000,34841,-0.001030,효성
2021-11-24,97000,98000,96800,97300,37727,0.003093,효성


In [ ]:
import plotly.express as px

fig = px.line(cor_price_data, x=cor_price_data.index, y="Close", color='cor_Name',title = '선택 종목 가격 추이')
fig.show()

In [ ]:
pivot_cor_data = cor_price_data.pivot_table(index = cor_price_data.index, columns = 'cor_Name', values = 'Close')
pivot_cor_data.head()

cor_Name,맥쿼리인프라,메리츠화재,삼성전자,쌍용C&E,한양이엔지,효성
Date,,,,,,
2018-01-02,8220,23350,51020,4086,13900,82259
2018-01-03,8230,23350,51620,4119,13850,87289
2018-01-04,8180,22700,51080,3999,13600,84034
2018-01-05,8170,23250,52120,4020,13550,84626
2018-01-08,8140,23000,52020,4382,13850,81075


In [ ]:
# new_price/old_price - 1 
returns = pivot_cor_data.pct_change() # 가격 변화율 추이
returns

cor_Name,맥쿼리인프라,메리츠화재,삼성전자,쌍용C&E,한양이엔지,효성
Date,,,,,,
2018-01-02,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-03,0.001217,0.000000,0.011760,0.008076,-0.003597,0.061148
2018-01-04,-0.006075,-0.027837,-0.010461,-0.029133,-0.018051,-0.037290
2018-01-05,-0.001222,0.024229,0.020360,0.005251,-0.003676,0.007045
2018-01-08,-0.003672,-0.010753,-0.001919,0.090050,0.022140,-0.041961
...,...,...,...,...,...,...
2021-11-22,0.007117,0.010327,0.051966,0.014139,0.028302,0.007261
2021-11-23,0.003534,0.010221,0.005340,-0.011407,-0.018349,-0.001030
2021-11-24,0.000000,0.037099,-0.006640,-0.006410,0.003115,0.003093


In [ ]:
# 연간 공분산 -> 얼마나 함께 이동하는지 판단한다.
cov_matrix_annual = returns.cov() * 252
cov_matrix_annual

cor_Name,맥쿼리인프라,메리츠화재,삼성전자,쌍용C&E,한양이엔지,효성
cor_Name,,,,,,
맥쿼리인프라,0.012298,0.009327,0.004782,0.006783,0.010529,0.010815
메리츠화재,0.009327,0.117169,0.018383,0.023209,0.031714,0.034459
삼성전자,0.004782,0.018383,0.071807,0.016690,0.052312,0.025881
쌍용C&E,0.006783,0.023209,0.016690,0.171265,0.038680,0.047653
한양이엔지,0.010529,0.031714,0.052312,0.038680,0.175144,0.050073
효성,0.010815,0.034459,0.025881,0.047653,0.050073,0.166196


In [ ]:
# 변동성을 확인하기 위한 분산 기대값
port_variance = np.dot(종목비중.T, np.dot(cov_matrix_annual, 종목비중))
port_variance

0.03792250097437533

In [ ]:
# 포트폴리오의 변동성(표준편차)
port_volatility = np.sqrt(port_variance)
port_volatility

0.19473700463541932

In [ ]:
# 연간 수익률
portfolioSimpleAnnualReturn = np.sum(returns.mean()*종목비중) * 252
portfolioSimpleAnnualReturn

0.1497215104364242

In [ ]:
percent_var = str(round(port_variance, 2) * 100) + '%'
percent_vols = str(round(port_volatility, 2) * 100) + '%'
percent_ret = str(round(portfolioSimpleAnnualReturn, 2)*100)+'%'
print("Expected annual return : "+ percent_ret)
print('Annual volatility/standard deviation/risk : '+percent_vols)
print('Annual variance : '+percent_var)

Expected annual return : 15.0%
Annual volatility/standard deviation/risk : 19.0%
Annual variance : 4.0%


In [ ]:
# 최적화

In [ ]:
pip install PyPortfolioOpt

     |████████████████████████████████| 61 kB 9.2 MB/s 
     |████████████████████████████████| 2.8 MB 48.0 MB/s 
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.0.31
    Uninstalling cvxpy-1.0.31:
      Successfully uninstalled cvxpy-1.0.31


In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [ ]:
mu = expected_returns.mean_historical_return(pivot_cor_data)#returns.mean() * 252
S = risk_models.sample_cov(pivot_cor_data) #Get the sample covariance matrix

In [ ]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe() #Maximize the Sharpe ratio, and get the raw weights
cleaned_weights = ef.clean_weights() 
print(cleaned_weights) #Note the weights may have some rounding error, meaning they may not add up exactly to 1 but should be close
ef.portfolio_performance(verbose=True)

OrderedDict([('맥쿼리인프라', 0.93358), ('메리츠화재', 0.0), ('삼성전자', 0.02163), ('쌍용C&E', 0.04478), ('한양이엔지', 0.0), ('효성', 0.0)])
Expected annual return: 15.3%
Annual volatility: 10.9%
Sharpe Ratio: 1.22


(0.1530685346301766, 0.10903562978478515, 1.2204133171223728)

### 자신이 보유하고 있는 종목을 4~6개 정도 입력한 뒤 포트를 최적화 해보자.

# ETF - 연금저축 포트 구성해보기

In [ ]:
ETF_ticker = fdr.StockListing("ETF/KR").rename(columns={"Symbol": "티커"})
ETF_ticker.head()

,티커,Name
0,069500,KODEX 200
1,371460,TIGER 차이나전기차SOLACTIVE
2,252670,KODEX 200선물인버스2X
3,102110,TIGER 200
4,153130,KODEX 단기채권


In [ ]:
import pandas as pd 

# 구글 드라이브 경로에서 read하기 위한 경로로 수정합니다.
def chage_google_drive(drive_path):
  result_path = url2='https://drive.google.com/uc?id=' + drive_path.split('/')[-2]
  return result_path

# data의 공유 path를 불러옵니다.
ETF_price_path = 'https://drive.google.com/file/d/1uNDSMuPdAD8CqwO1q9_JPcrt9hfTQyxk/view?usp=sharing'

# 데이터를 read합니다.
ETF_OHLCV = pd.read_csv(chage_google_drive(ETF_price_path))

In [ ]:
ETF_OHLCV = ETF_OHLCV.merge(ETF_ticker, on = 'Name')
ETF_OHLCV.head()

,날짜,NAV,시가,고가,저가,종가,거래량,거래대금,기초지수,Name,티커_x,티커_y
0,2021-01-04,40615.17,39585,40615,39405,40540,12185235,489888903885,399.88,KODEX 200,069500,069500
1,2021-01-05,41230.17,40325,41150,40200,41150,9126676,370933192250,406.03,KODEX 200,069500,069500
2,2021-01-06,40835.69,41270,41680,40595,40750,15221599,625103057790,402.08,KODEX 200,069500,069500
3,2021-01-07,41709.04,41210,42040,41190,41700,8752407,364764290470,410.81,KODEX 200,069500,069500
4,2021-01-08,43651.13,42245,43785,42115,43545,11521190,494420112265,430.22,KODEX 200,069500,069500


In [ ]:
# 연금저축 종목 구성
ETF_DATA = ETF_OHLCV[(ETF_OHLCV['Name'] == 'KODEX WTI원유선물(H)') |
         (ETF_OHLCV['Name'] == 'KODEX 3대농산물선물(H)') |
         (ETF_OHLCV['Name'] == 'KODEX 구리선물(H)') |
         (ETF_OHLCV['Name'] == 'TIGER 금은선물(H)')|
         (ETF_OHLCV['Name'] == 'TIGER 단기통안채')|
         (ETF_OHLCV['Name'] == 'TIGER 미국나스닥100')|
         (ETF_OHLCV['Name'] == 'KODEX 차이나CSI300')|
         (ETF_OHLCV['Name'] == 'TIGER 미국달러단기채권액티브')] 

In [ ]:
# 원자재들 + 금 + 달러는 각각 어떻게 움직일까?
materials_data = ETF_OHLCV[(ETF_OHLCV['Name'] == 'KODEX WTI원유선물(H)') |
         (ETF_OHLCV['Name'] == 'KODEX 3대농산물선물(H)') |
         (ETF_OHLCV['Name'] == 'KODEX 구리선물(H)') |
         (ETF_OHLCV['Name'] == 'TIGER 금은선물(H)') |
          (ETF_OHLCV['Name'] == 'TIGER 미국달러단기채권액티브')]

In [ ]:
import plotly.express as px

fig = px.line(materials_data, x="날짜", y="종가", color = 'Name',title='원자재 ETF 비교')
fig.show()

- 구리와 농산물은 비슷하게 움직인다.

In [ ]:
ETF_DATA = ETF_OHLCV[
         (ETF_OHLCV['Name'] == 'TIGER 금은선물(H)')|
         (ETF_OHLCV['Name'] == 'TIGER 단기통안채')|
         (ETF_OHLCV['Name'] == 'KODEX 구리선물(H)') |
         (ETF_OHLCV['Name'] == 'TIGER 미국나스닥100')|
         (ETF_OHLCV['Name'] == 'KODEX 차이나CSI300')] 

In [ ]:
ETF_pivot = ETF_DATA.pivot_table(index = '날짜', columns = 'Name', values = '종가')
ETF_pivot

Name,KODEX 구리선물(H),KODEX 차이나CSI300,TIGER 금은선물(H),TIGER 단기통안채,TIGER 미국나스닥100
날짜,,,,,
2021-01-04,6690,14085,10415,100020,62535
2021-01-05,6675,14300,10505,100025,61970
2021-01-06,6800,14465,10505,100030,61640
2021-01-07,6835,14570,10365,100030,62005
2021-01-08,6910,14665,10315,100035,63420
...,...,...,...,...,...
2021-11-19,8045,14530,9865,100430,87970
2021-11-22,8010,14650,9775,100430,88255
2021-11-23,8040,14685,9575,100430,87030


- 주식 30%, 장기채 40%, 중기채 15%, 원자재 7.5%, 금 7.5%
    - 안전 자산 : 채권, 금, 달러
        - 중기채 1~5년
        - 장기채 5년이상
    - 위험자산 : 주식, 원자재
        - 원자재 : 원유, 금속, 농수산물 등
- 위험자산의 범위는 30%에서 70%
- 기본적인 60:40 포트폴리오

- 위험자산 70%
    - KODEX 구리선물(H) 7.5%
    - KODEX 차이나CSI300 31%
    - TIGER 미국나스닥100 31.5%
- 안전자산 30%
    - TIGER 금은선물(H) 7.5%
    - TIGER 단기통안채 22.5%

In [ ]:
종목리스트 = ETF_pivot.columns.to_list()
print(종목리스트)
종목비중 = np.array([0.075, 0.31, 0.075, 0.225, 0.31])

['KODEX 구리선물(H)', 'KODEX 차이나CSI300', 'TIGER 금은선물(H)', 'TIGER 단기통안채', 'TIGER 미국나스닥100']


In [ ]:
import plotly.graph_objects as go

# Create traces
fig = go.Figure()
for corp_columns in ETF_pivot.columns:
    
    fig.add_trace(go.Scatter(x=ETF_pivot.index, y=ETF_pivot[corp_columns],
                        mode='lines',
                        name=f'{corp_columns}'))

fig.show()

In [ ]:
# new_price/old_price - 1
returns = ETF_pivot.pct_change()
returns

Name,KODEX 구리선물(H),KODEX 차이나CSI300,TIGER 금은선물(H),TIGER 단기통안채,TIGER 미국나스닥100
날짜,,,,,
2021-01-04,NaN,NaN,NaN,NaN,NaN
2021-01-05,-0.002242,0.015264,0.008641,0.00005,-0.009035
2021-01-06,0.018727,0.011538,0.000000,0.00005,-0.005325
2021-01-07,0.005147,0.007259,-0.013327,0.00000,0.005921
2021-01-08,0.010973,0.006520,-0.004824,0.00005,0.022821
...,...,...,...,...,...
2021-11-19,0.030090,0.009028,0.000000,0.00000,0.016818
2021-11-22,-0.004351,0.008259,-0.009123,0.00000,0.003240
2021-11-23,0.003745,0.002389,-0.020460,0.00000,-0.013880


In [ ]:
# 연간 공분산
cov_matrix_annual = returns.cov() * 252
cov_matrix_annual

Name,KODEX 구리선물(H),KODEX 차이나CSI300,TIGER 금은선물(H),TIGER 단기통안채,TIGER 미국나스닥100
Name,,,,,
KODEX 구리선물(H),0.058340,0.003307,0.010381,-5.582602e-06,0.005674
KODEX 차이나CSI300,0.003307,0.030466,0.000368,-2.534290e-05,0.006358
TIGER 금은선물(H),0.010381,0.000368,0.021150,9.175250e-06,-0.000128
TIGER 단기통안채,-0.000006,-0.000025,0.000009,6.056761e-07,-0.000007
TIGER 미국나스닥100,0.005674,0.006358,-0.000128,-6.907087e-06,0.026809


In [ ]:
# 분산 기대값
port_variance = np.dot(종목비중.T, np.dot(cov_matrix_annual, 종목비중))
port_variance

0.007714491908174124

In [ ]:
port_volatility = np.sqrt(port_variance)
port_volatility

0.08783218036787042

In [ ]:
# 연간 수익률
portfolioSimpleAnnualReturn = np.sum(returns.mean()*종목비중) * 252
portfolioSimpleAnnualReturn

0.15549330802901812

In [ ]:
percent_var = str(round(port_variance, 2) * 100) + '%'
percent_vols = str(round(port_volatility, 2) * 100) + '%'
percent_ret = str(round(portfolioSimpleAnnualReturn, 2)*100)+'%'
print("Expected annual return : "+ percent_ret)
print('Annual volatility/standard deviation/risk : '+percent_vols)
print('Annual variance : '+percent_var)

Expected annual return : 16.0%
Annual volatility/standard deviation/risk : 9.0%
Annual variance : 1.0%


In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [ ]:
mu = expected_returns.mean_historical_return(ETF_pivot)
S = risk_models.sample_cov(ETF_pivot)

In [ ]:
ef = EfficientFrontier(mu, S)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights() 
print(cleaned_weights) 
ef.portfolio_performance(verbose=True)

OrderedDict([('KODEX 구리선물(H)', 0.14702), ('KODEX 차이나CSI300', 0.0), ('TIGER 금은선물(H)', 0.0), ('TIGER 단기통안채', 0.0), ('TIGER 미국나스닥100', 0.85298)])
Expected annual return: 43.7%
Annual volatility: 14.9%
Sharpe Ratio: 2.80


(0.4369257393377645, 0.14896097308275288, 2.798892426046036)

### 자신이 보유하고 있는 종목(ETF) or 임의의 5종목을 선정하여 포트를 구성해보자.